In [1]:
import dropbox
import datetime
import schedule
import numpy as np
import string
import requests

from threading import Timer
import time
import sys
import os

import csv
import sqlite3


import math
import random
import matplotlib as mpl
import matplotlib.pyplot as plt

import matplotlib.dates as mdate

from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import FormatStrFormatter


import codecs

In [3]:
class EBikeData_Getting:
        
    def EBike(x_t,y_hP):
        # Dropbox access token    
        dbx = dropbox.Dropbox('lY_d3DAmzgAAAAAAAAAAs2SoAOwHmVVqP5ozJcw4sDBPvIdzSSwISAjUuy1eDLt4')

        # Set parameters for downloading the file 
        # Saving path
        save_path = 'C:\\FINAL_PROJECT\\EBike_Files\\Data.db'
        # File location in dropbox    
        path = '/EBIKE.db'
        
         # current time
        timestamp = datetime.datetime.now()
        # time gap
        delta = datetime.timedelta(minutes = 1)
        # time where the IBI file has been bulit
        filetime = timestamp - delta

        # Re-formatting: datetime->String
        timeframe = filetime.strftime('%Y-%m-%d %H:%M')
        #res = dbx.files_list_folder(path,None)
        #print (res)
        # try download
        try:
            dbx.files_download_to_file(save_path,path,None)   
        except:
                print('Database current is not exsit')
                pass
        else:
            print("Successful Downloading database file from Dropbox, overwriting " + save_path + "...")
    
            # Save voltage of the motor and data which averaged
            conn = sqlite3.connect(save_path)
            cursor = conn.cursor()
            
            #timeframe = '2019-07-19 19:28'

            res_findData=cursor.execute("SELECT * FROM bikeData WHERE time LIKE '%%%s%%'" %timeframe) # find "key words" in column time of table bikeData

            if cursor.fetchone() is None:   # No corresponding data to the time, need Optimize
                print('Not Found')
                pass
            else:                   # Have such data
                rows = cursor.fetchall()

                sum_hP = 0
                sum_ThrOut = 0
                count = 0

                for row in rows:
                    sum_hP += row[9]
                    sum_ThrOut += row[12]
                    count += 1

            conn.close()

            if count == 0:
                pass
            
            else:
                ave_hP = sum_hP/count
                ave_ThrOut = sum_ThrOut/count
                # export the data processed into file "export_EBikedata.csv"

                x_t.append(timeframe)
                y_hP.append(ave_hP)
                
                fig = HR_Getting.draw(x_t, y_hP)
                
                with open("export_EBikedata.csv", "a", newline='') as f:
                    headers = ['time', 'average humanPower', 'average ThrottleOutput' ]
                    writer = csv.writer(f)

                    if not os.path.getsize("export_EBikedata.csv"):

                        # print('no headers')
                        writer.writerow(headers) # file doesn't exist yet, write a header
                    else:
                        # print('have headers')
                        writer.writerow([timeframe,ave_hP,ave_ThrOut])
                        # Remove duplicates
                        # pd.read_csv("export_EBikedata.csv").drop_duplicates(subset ="First Name",keep = False, inplace = True)
                        f.close()
    def StartEBike():
        x_t = []
        y_hP = []
        schedule.every(1).minutes.do(EBikeData_Getting.EBike,x_t,y_hP)
        while True:
            schedule.run_pending()
            
    def draw(x_t, y_hP):
        plt.clf()
        plt.plot(x, y, '-r')
        plt.tick_params(axis='x', rotation=90)
        plt.title('bpm verus time')
        plt.xlabel('time')
        plt.ylabel('human power')
        plt.pause(0.01)  # pause a bit so that plots are updated
            
if __name__ == "__main__":
    schedule.clear()
    EBikeData_Getting.StartEBike()
    print ('here')


Successful Downloading database file from Dropbox, overwriting C:\FINAL_PROJECT\EBike_Files\Data.db...


DatabaseError: database disk image is malformed

In [18]:
#View the tables inside the Data.db
conn = sqlite3.connect( 'C:\\FINAL_PROJECT\\EBike_Files\\Data.db')
cursor = conn.cursor()
sql = """select name from sqlite_master where type='table' order by name"""
cursor.execute(sql)
result = cursor.fetchall()
print (result)
conn.close()

[('android_metadata',), ('bikeData',), ('bikeLocation',), ('breathingControl',), ('caloriesControl',), ('commandSent',), ('cooperativeBreathingControl',), ('heartRate',), ('mBandCaloriesBurned',), ('mTarget',), ('motorFilter',), ('sqlite_sequence',), ('trafficLight',)]


In [19]:
# headers of tables in .db file
conn = sqlite3.connect( 'C:\\FINAL_PROJECT\\EBike_Files\\Data.db')
cursor = conn.cursor()
sql = """pragma table_info(bikeData)"""
cursor.execute(sql)
result = cursor.fetchall()
print (result)
conn.close()

[(0, '_id', 'INTEGER', 0, None, 1), (1, 'time', 'TIMESTAMP', 0, 'CURRENT_TIMESTAMP', 0), (2, 'batteryEnergy', 'FLOAT', 0, None, 0), (3, 'voltage', 'FLOAT', 0, None, 0), (4, 'current', 'FLOAT', 0, None, 0), (5, 'speed', 'FLOAT', 0, None, 0), (6, 'distance', 'FLOAT', 0, None, 0), (7, 'temperature', 'FLOAT', 0, None, 0), (8, 'RPM', 'FLOAT', 0, None, 0), (9, 'humanPower', 'FLOAT', 0, None, 0), (10, 'torque', 'FLOAT', 0, None, 0), (11, 'throttleIn', 'FLOAT', 0, None, 0), (12, 'throttleOut', 'FLOAT', 0, None, 0), (13, 'acceleration', 'FLOAT', 0, None, 0), (14, 'flag', 'STRING', 0, None, 0)]


In [239]:
# Save voltage of the motor and data which averaged
conn = sqlite3.connect( 'C:\\FINAL_PROJECT\\EBike_Files\\Data.db')
cursor = conn.cursor()
timeframe = '2019-07-19 19:28'

res_findData=cursor.execute("SELECT * FROM bikeData WHERE time LIKE '%%%s%%'" %timeframe) # find "key words" in column time of table bikeData

if cursor.fetchone() is None:   # No corresponding data to the time, need Optimize
    print('Not Found')
    pass
else:                   # Have such data
    rows = cursor.fetchall()
    
    sum_hP = 0
    sum_ThrOut = 0
    count = 0
        
    for row in rows:
        sum_hP += row[9]
        sum_ThrOut += row[12]
        count += 1
        
conn.close()

if count == 0:
    pass
else:
    ave_hP = sum_hP/count
    ave_ThrOut = sum_ThrOut/count
    # export the data processed into file "export_EBikedata.csv"

    with open("export_EBikedata.csv", "a", newline='') as f:
        headers = ['time', 'average humanPower', 'average ThrottleOutput' ]
        writer = csv.writer(f)

        if not os.path.getsize("export_EBikedata.csv"):
            
            # print('no headers')
            writer.writerow(headers) # file doesn't exist yet, write a header
        else:
            # print('have headers')
            writer.writerow([timeframe,ave_hP,ave_ThrOut])
            # Remove duplicates
            # pd.read_csv("export_EBikedata.csv").drop_duplicates(subset ="First Name",keep = False, inplace = True)
            f.close()



In [5]:
        timestamp = datetime.datetime.now()
        # time gap
        delta = datetime.timedelta(minutes = 1)
        # time where the IBI file has been bulit
        filetime = timestamp - delta

        # Re-formatting: datetime->String
        timeframe = filetime.strftime('%Y-%m-%d %H:%M')
        print(timeframe)

2019-08-16 20:10


In [4]:
        dbx = dropbox.Dropbox('lY_d3DAmzgAAAAAAAAAAs2SoAOwHmVVqP5ozJcw4sDBPvIdzSSwISAjUuy1eDLt4')

        # Set parameters for downloading the file 
        # Saving path
        save_path = 'C:\\FINAL_PROJECT\\EBike_Files\\Data.db'
        # File location in dropbox    
        path = '/EBIKE.db'
        #res = dbx.files_list_folder(path,None)
        #print (res)
        # try download
        dbx.files_download_to_file(save_path,path,None)   

FileMetadata(name='EBike.db', id='id:HDe7egdlNmAAAAAAAAAJvw', client_modified=datetime.datetime(2019, 8, 16, 18, 47, 5), server_modified=datetime.datetime(2019, 8, 16, 18, 47, 5), rev='01590406aa75a03000000016b5ec0a0', size=17559552, path_lower='/ebike.db', path_display='/EBike.db', parent_shared_folder_id=None, media_info=None, symlink_info=None, sharing_info=None, property_groups=None, has_explicit_shared_members=None, content_hash='d60de019835fc1fb18c1310cc7b7a404a0e53fbd5b739f91b34f1cf36d3ccef3')